In [16]:
import pandas as pd
import deltalake
from deltalake.writer import write_deltalake
from deltalake import DeltaTable
import os
os.makedirs('data/temps_lake', exist_ok=True)
print(deltalake.__version__)

0.25.2


In [7]:
# First, read the CSV without date parsing
landtemps = pd.read_csv('data/landtempssample.csv',
                      names=['stationid', 'year', 'month', 'avgtemp', 'latitude',
                             'longitude', 'elevation', 'station', 'countryid', 'country'],
                      skiprows=1)

# Then create the datetime column after reading
landtemps['month_year'] = pd.to_datetime(landtemps[['month', 'year']].assign(day=1))

# Optionally drop the original separate columns if no longer needed
landtemps = landtemps.drop(['year', 'month'], axis=1)

landtemps.shape


(100000, 9)

In [8]:

landtemps.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   stationid   100000 non-null  object        
 1   avgtemp     85554 non-null   float64       
 2   latitude    100000 non-null  float64       
 3   longitude   100000 non-null  float64       
 4   elevation   100000 non-null  float64       
 5   station     100000 non-null  object        
 6   countryid   100000 non-null  object        
 7   country     99995 non-null   object        
 8   month_year  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 6.9+ MB


In [19]:
print(landtemps.dtypes)


stationid             object
avgtemp              float64
latitude             float64
longitude            float64
elevation            float64
station               object
countryid             object
country               object
month_year    datetime64[ns]
dtype: object


In [20]:
landtemps["month_year"] = landtemps["month_year"].dt.tz_localize("UTC")

In [22]:
write_deltalake(
    'data/temps_lake',
    landtemps,
)


In [23]:
tempsdelta = DeltaTable('data/temps_lake', version=0)
type(tempsdelta)


deltalake.table.DeltaTable

In [25]:
tempsdfv1 = tempsdelta.to_pandas()
tempsdfv1.shape


(100000, 9)

In [26]:
write_deltalake('data/temps_lake', landtemps.head(1000),
                mode='overwrite')


In [27]:
tempsdfv2 = DeltaTable('data/temps_lake', version=1).to_pandas()
tempsdfv2.shape







(1000, 9)

In [28]:
write_deltalake('data/temps_lake', landtemps.head(1000), mode='append')
tempsdfv3 = DeltaTable('data/temps_lake', version=2).to_pandas()
tempsdfv3.shape







(2000, 9)

In [29]:
DeltaTable('data/temps_lake', version=0).to_pandas().shape


(100000, 9)